In [14]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir("/content/gdrive/MyDrive/DISSERTATION/SFC-CAE-Adaptive")
print(os.listdir())

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
['plotloss.py', 'LICENSE', 'get_FPC_data_CG.sh', 'README.md', 'requirements.txt', 'vtktools.py', 'get_FPC_data_DG.sh', 'tSNE.py', 'get_data_slugflow.sh', 'environment.yml', 'command_train.py', 'space_filling_decomp_new.f90', 'parameters.ini', '.github', 'Colab_Notebooks', '__pycache__', 'sfc_cae', 'tests', 'pics', 'SFC_CAE.egg-info', 'LatexTable.txt', 'SFC_CAE__block_latent', 'Block', 'SFC_CAE__block_latent16_3fc_True_nn.mp4', 'SFC_CAE__block_latent_8_1fc.mp4', 'SFC_CAE__block_latent16_3fc_False_nn.mp4', 'SFC_CAE__block_latent16_2fc_True_nn.mp4', 'SFC_CAE__block_latent16_2fc_False_nn.mp4', 'SFC_CAE__block_latent16_1fc_True_nn.mp4', 'SFC_CAE__block_latent16_1fc_False_nn.mp4', 'SFC_CAE__block_latent16_0fc_True_nn.mp4', 'SFC_CAE__block_latent16_0fc_False_nn.mp4', 'SFC_CAE__block_latent8_3fc_True_nn.mp4', 'SFC_CAE__block_latent8_3fc_False_nn.mp4', 'SFC_CAE__b

In [15]:
modulelist = !pip list                         #This is just to check whether the requirements are already installed
if str(modulelist).find("vtk") == -1:          #If vtk is already installed don't start the installation process because it will take a couple minutes to verify that everything is there.
  !pip install -e .

In [16]:
from sfc_cae import *

In [17]:
os.chdir("..")

In [18]:
#Importing a bunch of stuff from MeshCNN
os.chdir("./MeshCNN")
print(os.listdir())
import models
import options.train_options
os.chdir("./util")
import mesh_viewer
os.chdir("..")
os.chdir("./models")
import networks
os.chdir("./layers")
import mesh
import mesh_pool
import mesh_conv
import mesh_prepare
os.chdir("../../../")

['.git', '.gitignore', '.travis.yml', 'LICENSE', 'README.md', 'data', 'docs', 'environment.yml', 'models', 'options', 'scripts', 'test.py', 'train.py', 'util', 'cache']


In [19]:
#Importing a bunch more libraries
import numpy as np
import matplotlib.pyplot as plt
import torch  # Pytorch
import torch.nn as nn  # Neural network module
import torch.nn.functional as fn  # Function module
from torchvision import datasets  # Datasets from torchvision
from torchvision import transforms  # Transforms from torchvision
from torch.utils.data import DataLoader
from matplotlib import animation
import sys
import progressbar
import copy

In [20]:
!pip install pycm livelossplot
%pylab inline
from livelossplot import PlotLosses

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['tri', 'copy', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [21]:
import random 
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True

device = 'cuda'  # Set out device to GPU

print('Cuda installed, running on GPU!')  # print sentence

Cuda installed, running on GPU!


In [22]:
#Opt class from MeshCNN, to create a "Mesh" class you need an option class for data augmentation purposes, hard to get mesh loading to work otherwise
class opt:
  num_aug = 1

In [23]:
#Creating my flow past cylinder mesh
fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
fpcmesh.init_history()

In [24]:
#Extracting the edge features (the 5 dimensional dihedral angle, etc that they talk about in the paper)
mfeat = mesh_prepare.extract_features(fpcmesh)

In [47]:
#Creating my GCAE class

class GCAE(nn.Module):
    """Network for fully-convolutional tasks (segmentation)
    """
    def __init__(self, pools, down_convs, up_convs, linear = [128] ,blocks=0, transfer_data=True, verbose = False):
        super(GCAE, self).__init__()
        self.transfer_data = transfer_data
        self.encoder = networks.MeshEncoder(pools, down_convs, blocks=blocks)

        unrolls = pools[:-1].copy()
        unrolls.reverse()
        self.decoder = networks.MeshDecoder(unrolls, up_convs, blocks=blocks, transfer_data=transfer_data)
        self.fc = nn.Sequential(nn.Linear(pools[-1],linear[0]),nn.Linear(linear[0],pools[-1]))
        self.verbose = verbose

    def forward(self, x, meshes):

        #Concatenating edge geometrical features extracted by MeshCNN with my own x and y velcity features
        
        if self.verbose: print("Before concatenating:", x.shape, mfeat.shape)
        x = torch.cat((x, torch.Tensor(mfeat).unsqueeze(0).repeat(x.shape[0],1,1).to("cuda")),1)
        if self.verbose: print("After concatenating:", x.shape)

        #Encoding with Graph Convolutional Layers
        fe, before_pool = self.encoder((x, meshes))
        
        #Going through fully connected layers
        fe = self.fc(fe)

        if self.verbose: print("After fc:", fe.shape)

        fe = self.decoder((fe, meshes), before_pool)
        return fe

    def __call__(self, x, meshes):
        return self.forward(x, meshes)

In [48]:
#Creating our GCAE
GCAE = GCAE([10421,4096,2048,1024],[7,8,8,4,2],[2,4,8,8,2])

In [49]:
train_set = torch.load('./TENS/train_tensor.pt', map_location=torch.device('cpu'))
valid_set = torch.load('./TENS/valid_tensor.pt', map_location=torch.device('cpu'))[:50,:,:]  #Set to 50 because it crashes memory otherwise
test_set = torch.load('./TENS/test_tensor.pt', map_location=torch.device('cpu'))

In [50]:
#Standardizing our data (function borrowed from Yu Jin's SFC-CAE library)
train_set, k, b = standardlize_tensor(train_set)
valid_set, k2, b2 = standardlize_tensor(valid_set)
test_set, k3, b3 = standardlize_tensor(test_set)

In [51]:
#Batch size unfortunately can only be 8 on a 24GB GPU (that's how memory consuming it is!)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=8, shuffle=True, num_workers=0)

In [53]:
#Creating an fpcmesh to copy throughout our code
import copy
fpcmeshinitial = mesh.Mesh("fpc3.obj",opt=opt)

In [54]:
def train(autoencoder, optimizer, criterion, dataloader):
  counter = 0
  autoencoder.train()
  train_loss, data_length = 0, len(dataloader.dataset)
  for batch in dataloader:
      counter +=1
      meshes = []
      for i in range(dataloader.batch_size):
        fpcmesh = copy.deepcopy(fpcmeshinitial)
        fpcmesh.init_history()
        meshes.append(fpcmesh)
      # fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
      # fpcmesh.init_history()
      batch = batch.to(device)  # Send batch of images to the GPU
      optimizer.zero_grad()  # Set optimiser grad to 0
      x_hat = autoencoder(batch,meshes)  # Generate predicted images (x_hat) by running batch of images through autoencoder
      MSE = criterion(batch, x_hat)  # Calculate MSE loss
      print(MSE,counter)
      MSE.backward()  # Back-propagate
      optimizer.step()  # Step the optimiser
      train_loss += MSE * batch.size(0)
      
      for m in meshes:
        del m.history_data
        del m

  return train_loss / data_length  # Return MSE

def validate(autoencoder, optimizer, criterion, dataloader):
    autoencoder.eval()
    validation_loss, data_length = 0, len(dataloader.dataset)
    for batch in dataloader:
        with torch.no_grad():
          meshes = []
          for i in range(dataloader.batch_size):
            fpcmesh = copy.deepcopy(fpcmeshinitial)
            fpcmesh.init_history()
            meshes.append(fpcmesh)
          batch = batch.to(device)  # Send batch of images to the GPU
          x_hat = autoencoder(batch,meshes)  # Generate predicted images (x_hat) by running batch of images through autoencoder
          MSE = criterion(batch, x_hat)  # Calculate MSE loss
          validation_loss += MSE * batch.size(0)
        
          for m in meshes:
            del m.history_data
            del m

    return validation_loss / data_length   # Return MSE

In [42]:
def train_model(autoencoder, batch_size=8, n_epochs = 10, visualize=True, lr = None):
  set_seed(42)
  autoencoder = autoencoder.to(device)
  if lr is not None:
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr = lr)
  else:
    optimizer = torch.optim.Adam(autoencoder.parameters())

  # we choose the MSE to be the loss function 
  criterion = nn.MSELoss()
  
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
  valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
  
  # do livelossplot if visualize turned-on
  if visualize:
      liveloss = PlotLosses()

  for epoch in range(n_epochs):
    train_MSE = train(autoencoder, optimizer, criterion, train_loader)
    # validation_MSE = validate(autoencoder, optimizer, criterion, valid_loader)
    print("eppoch %d starting......"%(epoch))
    
    # do livelossplot if visualize turned-on 
    if visualize: 
      logs = {}

      logs['' + 'log loss'] = train_MSE.item()
      # logs['val_' + 'log loss'] = validation_MSE.item()

      liveloss.update(logs)
      liveloss.draw()

    torch.save(autoencoder.state_dict, "Latesttanh"+str(epoch+10)+".pth")
    
  return autoencoder

In [43]:
# dicto = torch.load("./Latest10.pth")

In [44]:
# GCAE.load_state_dict(dicto())

In [55]:
autoencoder = train_model(GCAE,n_epochs=10, lr = 1e-2)

torch.Size([8, 2, 10421])
Before going through conv layer 1: torch.Size([8, 7, 10421])
After going through conv layer1: torch.Size([8, 8, 4096])
Before going through conv layer 2: torch.Size([8, 8, 4096])
After going through conv layer2: torch.Size([8, 8, 2048])
Before going through conv layer 3: torch.Size([8, 8, 2048])
After going through conv layer3: torch.Size([8, 4, 1024])
Before going through conv layer 4: torch.Size([8, 4, 1024])
After going through conv layer4: torch.Size([8, 2, 1024])
DECODER. Before going through conv layer 1: torch.Size([8, 2, 1024])
DECODER. After going through conv layer1: torch.Size([8, 4, 2048])
DECODER. Before going through conv layer 2: torch.Size([8, 4, 2048])
DECODER. After going through conv layer2: torch.Size([8, 8, 4096])
DECODER. Before going through conv layer 3: torch.Size([8, 8, 4096])
DECODER. After going through conv layer3: torch.Size([8, 8, 10421])
saving
tensor(1.1742, device='cuda:0', grad_fn=<MseLossBackward>) 1
torch.Size([8, 2, 10421]

KeyboardInterrupt: ignored

In [ ]:
meshes = []
for i in range(99):
  fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
  fpcmesh.init_history()
  meshes.append(fpcmesh)

In [ ]:
test_out = AEFC2(test_set[:10],meshes[:10])

In [ ]:
test_set.shape()

In [ ]:
full_tensor, coords, cells = read_in_files(data_path, vtu_fields = vtu_fields)

In [ ]:
vtu_file = meshio.read("fpc_1.vtu")
coords = vtu_file.points
cells = vtu_file.cells_dict

Once we have our 10421x2 vector we need to turn it back from edges into vertices. To do that we reverse solve the linear system with a 10421x3571 matrix that has value 0.5 whenever a vertex was averaged to create the value of an edge. Unfortunately numpy.solve only supports square matrices but the lstsq method (which finds an x which minimises the difference between Ax and b) works just fine

In [ ]:
M = np.zeros((10421,3571))

In [ ]:
for i in range(len(fpcmesh.edges)):
  M[i,fpcmesh.edges[i,0]] = 0.5
  M[i,fpcmesh.edges[i,1]] = 0.5

In [ ]:
test_out[0].sh

In [ ]:
M.shape

In [ ]:
sol = np.linalg.lstsq(M,test_out[5].permute(1,0).cpu().detach().numpy())

In [ ]:
sol2 = np.linalg.lstsq(M,test_set[5].permute(1,0).cpu().detach().numpy())

In [ ]:
sol[0].shape

In [ ]:
point_data = {}
point_data.update({"Velocity":sol[0]}) #Change to sol2[0] to save the other vector
mesh = meshio.Mesh(coords,cells,point_data)
mesh.write("reconstructedgcae" + str(i) + ".vtu")

In [ ]:
fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
fpcmesh.init_history()
meshes.append(fpcmesh)